# Imports and Configs

The configs are stored in `src/model/modelConfig.py`

You can either use them as they are or modify them. alternativly you can specify your own confgis here in this file.

You need the following variables:
- DEVICE
- EPOCH_SIZE
- HIDDEN_SIZE
- LR

In [6]:
from model.modelConfig import *
from model.StockModel import StockPredictor
from StocksDataset import StocksDataset
from torch import utils, nn, optim, no_grad

# Data Collection

Choose your prefered stock price to begin data collection.

The `StockDatset` class fetches data from Yahoo Finace using `yfinance` library.
<br>
<br>
<br>
You can modify the following line:
```python
dataset = StocksDataset(sTickerSymbol="stock-symbol")
```
Alternatively, if your data is stored on your machine, you can import it like so:
```python
dataset = StocksDataset(sPath="path/to/csv.csv")
```

In [15]:
dataset = StocksDataset(sPath="../data/data.csv")

trainData_UB = int(0.85* len(dataset))

trainDataset = utils.data.Subset(dataset, range(0, trainData_UB))
testDataset  = utils.data.Subset(dataset, range(trainData_UB, len(dataset)))

trainLoader = utils.data.DataLoader(trainDataset, batch_size=32, shuffle=True)
testLoader  = utils.data.DataLoader(testDataset, batch_size=32, shuffle=True)

In [8]:
model = StockPredictor(3, HIDDEN_SIZE, 1)

In [9]:
def train(_model: StockPredictor, _loader,_epochSize: int, _device, _lr=0.001,_lossFN=nn.SmoothL1Loss()):
    _model.to(_device)
    _model.train()
    _model.zero_grad()
    optimizer = optim.Adam(_model.parameters(), lr=_lr)

    TOTAL_LOSS = []

    for epoch in range(1, _epochSize+1):
        epochLoss  = 0.0
        samples    = 0.0

        for d,x,y in _loader:
            
            optimizer.zero_grad()
            
            x = x.to(_device)
            y = y.to(_device).unsqueeze(1)

            prediction = _model(x)
            loss       = _lossFN(prediction, y)
            
            loss.backward()
            optimizer.step()

            epochLoss += loss.item() * y.size(0)
            samples   +=  y.size(0)
            
        epochLoss /= samples
        TOTAL_LOSS.append(epochLoss)
        
        print(f"epoch #{epoch} -- loss: {epochLoss}")
    return TOTAL_LOSS

In [10]:
def eval(_model: StockPredictor, _device, _testLoader, _lossFN = nn.SmoothL1Loss()):
    _model.to(_device)
    _model.eval()
    # _model.zero_grad()
    
    SAMPLES   = 0
    TOTA_LOSS = 0
    
    with no_grad():
        for d, x, y in _testLoader:
            x = x.to(_device)
            y = y.to(_device).unsqueeze(1)

            prediction = _model(x)
            loss       = _lossFN(prediction, y)

            TOTA_LOSS += loss.item() * y.size(0)
            SAMPLES   += y.size(0)

    acc = TOTA_LOSS / SAMPLES
    print(f"accuracy: {acc*100:.2f}%")
    return acc

In [ ]:
train(_model=model,
      _loader=trainLoader,
      _epochSize=EPOCH_SIZE,
      _device=DEVICE,
      _lr=LR)